https://adventofcode.com/2019/day/6
# --- Day 6: Universal Orbit Map ---
You've landed at the Universal Orbit Map facility on Mercury. Because navigation in space often involves transferring between orbits, the orbit maps here are useful for finding efficient routes between, for example, you and Santa. You download a map of the local orbits (your puzzle input).

Except for the universal Center of Mass (COM), every object in space is in orbit around exactly one other object. An orbit looks roughly like this:
```
                  \
                   \
                    |
                    |
AAA--> o            o <--BBB
                    |
                    |
                   /
                  /
```
In this diagram, the object BBB is in orbit around AAA. The path that BBB takes around AAA (drawn with lines) is only partly shown. In the map data, this orbital relationship is written AAA)BBB, which means "BBB is in orbit around AAA".

Before you use your map data to plot a course, you need to make sure it wasn't corrupted during the download. To verify maps, the Universal Orbit Map facility uses orbit count checksums - the total number of direct orbits (like the one shown above) and indirect orbits.

Whenever A orbits B and B orbits C, then A indirectly orbits C. This chain can be any number of objects long: if A orbits B, B orbits C, and C orbits D, then A indirectly orbits D.

What is the total number of direct and indirect orbits in your map data?


In [8]:
test1 = """COM)B
B)C
C)D
D)E
E)F
B)G
G)H
D)I
E)J
J)K
K)L
"""


def parse_orbits(orbits):
    """Separate a multi-line string into a dict of pointers. 
    
    Each row of the string:  A)B 
    becomes a pointer from outer (B) -> inner (A)
    like so:  {'B':'A'}
    """
    lines = [x.split(')') for x in orbits.split('\n') if x]
    return {x[1]: x[0] for x in lines}

test_orbits = parse_orbits(test1)
test_orbits

{'B': 'COM',
 'C': 'B',
 'D': 'C',
 'E': 'D',
 'F': 'E',
 'G': 'B',
 'H': 'G',
 'I': 'D',
 'J': 'E',
 'K': 'J',
 'L': 'K'}

In [59]:
def orbit_count(orbits):
    """Count the total number of direct & indirect orbits in an orbital system.
    
    orbits = dictionary output from parse_orbits()
    """
    count = 0
    for speck in orbits:
        trace = speck
        count += 1
        while (orbits[trace] != 'COM'):
            count += 1
            trace = orbits[trace]
    return count


assert orbit_count(test_orbits) == 42

In [60]:
with open('../data/06a.txt', 'r') as f:
    raw_orbits = "".join(line for line in f)
    
orbits = parse_orbits(raw_orbits)
orbit_count(orbits)  # 171213

171213

### --- Part Two ---
Now, you just need to figure out how many orbital transfers you (YOU) need to take to get to Santa (SAN).

You start at the object YOU are orbiting; your destination is the object SAN is orbiting. An orbital transfer lets you move from any object to an object orbiting or orbited by that object.
```
                          YOU
                         /
        G - H       J - K - L
       /           /
COM - B - C - D - E - F
               \
                I - SAN
```

In [62]:
def orbit_path(speck, orbits=orbits):
    """Trace the path from a starting point back to 'COM'.
          
            G - H       J - K
           /           /
    COM - B - C - D - E
    
    speck = 'H'
    orbits = dictionary output from parse_orbits()
    
    path = ['H', 'G', 'B', 'COM']
    """
    path = []
    trace = speck
    
    while (orbits[trace] != 'COM'):
        path.append(trace)
        trace = orbits[trace]
        
    path.append('COM')    
    return path


def orbital_jumps(A, B):
    """Count the jumps to nearest shared neighbor, return the total jumps.
    
    A & B = list output from orbit_path()   
    ex)  ['H', 'G', 'B', 'COM']
    """
    # subtract 1 because it counts jumps, not planets
    A_path = len(A) - len(set(A) & set(B)) - 1
    B_path = len(B) - len(set(A) & set(B)) - 1
    return A_path + B_path


you = orbit_path('YOU')
santa = orbit_path('SAN')
orbital_jumps(you, santa)  # 292

292